In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datasetsforecast.long_horizon import LongHorizon

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, SOFTS, TSMixer

from utilsforecast.losses import mae, mse
from utilsforecast.evaluation import evaluate

D:\Anaconda\envs\neuralforecast-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_data(name):
    if name == "ettm1":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == "ettm2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == 'etth1':
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'
    elif name == "etth2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'

    return Y_df, val_size, test_size, freq

In [3]:
Y_df, val_size, test_size, freq = load_data('ettm1')

horizon = 96

models = [
    SOFTS(
        h=horizon, 
        input_size=3*horizon, 
        n_series=7,
        hidden_size=512,
        d_core=512,
        e_layers=2,
        d_ff=1024,
        use_norm=True,
        max_steps=1000, 
        early_stop_patience_steps=3),
    TSMixer(h=horizon, input_size=3*horizon, n_series=7, max_steps=1000, early_stop_patience_steps=3),
    NHITS(h=horizon, input_size=3*horizon, max_steps=1000, early_stop_patience_steps=3),
]

nf = NeuralForecast(models=models, freq=freq)
nf_preds = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=None)
nf_preds = nf_preds.reset_index()

ettm1_evaluation = evaluate(df=nf_preds, metrics=[mae, mse], models=['SOFTS', 'TSMixer', 'NHITS'])

INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                   | Params
---------------------------------------------------------
0 | loss          | MAE                    | 0     
1 | padder        | ConstantPad1d          | 0     
2 | scaler        | TemporalNorm           | 0     
3 | enc_embedding | DataEmbedding_inverted | 147 K 
4 | encoder       | TransEncoder           | 4.7 M 
5 | projection    | Linear                 | 49.2 K
-------------------------------------

Epoch 99: 100%|█████████████████| 1/1 [00:00<00:00,  5.65it/s, v_num=54, train_loss_step=0.332, train_loss_epoch=0.355]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 99: 100%|█| 1/1 [00:03<00:00,  3.63s/it, v_num=54, train_loss_step=0.332, train_loss_epoch=0.355, valid_loss=0.42
Epoch 199: 100%|█| 1/1 [00:00<00:00,  5.88it/s, v_num=54, train_loss_step=0.352, train_loss_epoch=0.358, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 199: 100%|█| 1/1 [00:03<00:00,  3.41s/it, v_num=54, train_loss_step=0.352, train_loss_epoch=0.358, valid_loss=0.4
Epoch 299: 100%|█| 1/1 [00:00<00:00,  5.68it/s, v_num=54, train_loss_step=0.362, train_loss_epoch=0.329, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                     

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.25s/it]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                     | Params
-----------------------------------------------------------
0 | loss          | MAE                      | 0     
1 | padder        | ConstantPad1d            | 0     
2 | scaler        | TemporalNorm             | 0     
3 | norm          | ReversibleInstanceNorm1d | 14    
4 | mixing_layers | Sequential               | 184 K 
5 | out           | Linear                   | 27.7 K
-----------------------------------------------------------
212 K     Trainable params
0         Non-trainable params
212 K     Total params
0.849     Total estimated model params size (MB)


...


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.29s/it]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 3.2 M 
-----------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.759    Total estimated model params size (MB)



Epoch 99: 100%|█████████████████| 1/1 [00:00<00:00,  1.78it/s, v_num=58, train_loss_step=0.327, train_loss_epoch=0.335]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 99: 100%|█| 1/1 [00:02<00:00,  2.68s/it, v_num=58, train_loss_step=0.327, train_loss_epoch=0.335, valid_loss=0.40
Epoch 199: 100%|█| 1/1 [00:00<00:00,  1.99it/s, v_num=58, train_loss_step=0.299, train_loss_epoch=0.314, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 199: 100%|█| 1/1 [00:02<00:00,  2.60s/it, v_num=58, train_loss_step=0.299, train_loss_epoch=0.314, valid_loss=0.4
Epoch 299: 100%|█| 1/1 [00:00<00:00,  2.42it/s, v_num=58, train_loss_step=0.296, train_loss_epoch=0.299, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                    

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.86s/it]


In [4]:
ettm1_evaluation = ettm1_evaluation.drop(['unique_id'], axis=1)\
                                   .groupby('metric')\
                                   .mean()\
                                   .reset_index()

ettm1_evaluation

,metric,SOFTS,TSMixer,NHITS
0,mae,0.345241,0.334170,0.347486
1,mse,0.306944,0.292147,0.317878
